In [1]:
#import require packages
import cv2
import tensorflow as tf
from keras.models import model_from_json
import numpy as np
import time

In [2]:
# load json and create model
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = tf.keras.models.model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("model/emotion_model.h5")

In [6]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Neutral", 6: "Surprised"}

# start the webcam feed
cap = cv2.VideoCapture(0)


# The countdown duration (in seconds)
countdown_duration = 15  

# The specific facial expression to detect (e.g., "anger" or "fear")
angry_expression = "Angry"
fearful_expression = "Fearful"

# Initialize variables
start_time = None
previous_expression = None
countdown_active = False

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
  
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=4)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        cropped_gray_face = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(cropped_gray_face, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        
        
     
    # countdown to check for a hostile situation.
    detected_expression = emotion_dict[maxindex]  

    if detected_expression != previous_expression:
        previous_expression = detected_expression
        countdown_active = False

    if detected_expression == angry_expression or detected_expression == fearful_expression:
        if not countdown_active:
            start_time = time.time()
            countdown_active = True
            if detected_expression == angry_expression:
                print(f"Countdown activated for {angry_expression} expression.")
            elif detected_expression == fearful_expression:
                print(f"Countdown activated for {angry_expression} expression.")    
        else:
            if time.time() - start_time >= countdown_duration:
                if detected_expression == angry_expression:
                    print(f"Alert triggered: {angry_expression} expression detected for {countdown_duration} seconds!")
                elif detected_expression == fearful_expression:
                    print(f"Alert triggered: {fearful_expression} expression detected for {countdown_duration} seconds!") 
    else:
        if countdown_active:
            print(f"Countdown terminated for {specific_expression} expression.")
            countdown_active = False

    time.sleep(1)  # Simulate checking every 1 second
   
    
    cv2.imshow('Surveillance System(Emotion Detection)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 29ms/step
Countdown activated for Angry expression.
1/1 [==============================] - 0s 19ms/step
Countdown activated for Angry expression.
1/1 [==============================] - 0s 31ms/step
Countdown activated for Angry expression.
1/1 [==============================] - 0s 16ms/step
Countdown activated for Angry expression.
1/1 [==============================] - 0s 16ms/step
Countdown activated for Angry expression.
1/1 [==============================] - 0s 16ms/step
Countdown activated for Angry expression.
1/1 [==============================] - 0s 31ms/step
Countdown activated for Angry expression.
1/1 [==============================] - 0s 31ms/step
